In [1]:
import math 

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, lower, upper, col, trim, year, month, dayofweek, translate, concat, regexp_replace, date_format, date_add, round
from pyspark.sql.types import IntegerType, StringType, IntegerType, DoubleType
from pyspark import SparkConf, SparkContext

from Functions.redShiftFunctions import redShiftQuery, redShiftWrite, redShiftRead
from Functions.loggerObject import createLogger
import config

#Create logger:
logger = createLogger('SparkJob_001', 'logFile_{}_.log'.format(datetime.now().strftime('%Y_%h')) )


# REPLACE ACCESS KEY AND SECRET KEY in config.py file!
conf = SparkConf() \
       .set("spark.hadoop.fs.s3a.access.key", config.s3aAccessKey) \
       .set("spark.hadoop.fs.s3a.secret.key", config.s3aSecretKey)\
       .set("spark.hadoop.fs.s3a.endpoint", config.s3aEndpoint)

spark = SparkSession \
        .builder.master("local[*]") \
        .appName("Main ETL for the Capstone Project")\
        .config(conf=conf)\
        .getOrCreate()


#Used to collect all dataframes.
tableNamesArray = []

In [4]:
#  ------  PORT LOCATION TABLE -----------
try:
    
    #Read the Portal Codes json file.
    textFile = spark.sparkContext.textFile(config.prtlCodes_file)
    dfPortCodes = spark.read.json(textFile)

    #Remove any rows that do not contain a name and drop duplicates.
    dfPortCodes = dfPortCodes.filter(~dfPortCodes.name.contains("No PORT") & ~dfPortCodes.name.contains("Collapsed") )
    dfPortCodes = dfPortCodes.drop_duplicates()

    #Split the name to columns municipality and statecode.
    nameSplit = split(dfPortCodes.name, ',')
    dfPortCodes = dfPortCodes.withColumn('municipality', trim (lower( nameSplit.getItem(0) ) ) ) \
                             .withColumn('statecode', trim(upper(nameSplit.getItem(1)))  ) 

    #Remove any rows without state code. Drop column name as its been split to two new columns.
    dfPortCodes = dfPortCodes.filter(~dfPortCodes.statecode.isNull()).drop("name")

    dfStateCode = spark.read.options(header=True, delimiter=',', inferSchema='True').csv(config.stateCodes_file)

    #Joins Portal Codes and State Codes together to have full state name.
    dfPortCodes = dfPortCodes.join(dfStateCode, dfPortCodes.statecode == dfStateCode["Alpha code"], how="left")

    #Drops Alpha code which is a duplicate of state code.
    dfPortCodes = dfPortCodes.drop("Alpha code").withColumn("state", lower(dfPortCodes.State))

    dfPortCodes.createOrReplaceTempView("PORTCODES")

    if config.limitDFtoTen:
        dfPortCodes = dfPortCodes.limit(10)

    #Appends Data frame with Key to be used at the end to insert into Database.
    tableNamesArray.append({'PORTCODES':dfPortCodes})
    
except Exception as e:
    logger.error(f'PORT LOCATION Table creations:  {e}')
else:
    logger.info(f'PORT LOCATION Dataframe size {dfPortCodes.count()}')


In [ ]:
#
#  ------  AIRPORTS TABLE
#

dfAirPorts = spark.read.option("header",True).csv(config.airportCodes_file)

# Our analysis will be set for US only.
dfAirPorts = dfAirPorts.filter( (dfAirPorts.iso_country == 'US') \
                               & (dfAirPorts.type != 'heliport') \
                               & (dfAirPorts.type != 'closed') \
                              )


latLongSplit = split(dfAirPorts.coordinates, ',')
isoRegSplit = split(dfAirPorts.iso_region, '-')

dfAirPorts = dfAirPorts.withColumn('longitude', latLongSplit.getItem(0).cast(DoubleType())) \
        .withColumn('latitude', latLongSplit.getItem(1).cast(DoubleType())) \
        .withColumn('statecode', upper(isoRegSplit.getItem(1))) \
        .withColumn('municipality', lower(col("municipality"))) \
        .withColumn('name', lower(col("name"))) \
        .withColumn('elevation_ft', col("elevation_ft").cast(IntegerType())) 


#Remove columns continent and iso_country since it is US only.
dfAirPorts = dfAirPorts.drop("iso_region", "continent", "coordinates", "gps_code", "local_code")

if config.limitDFtoTen:
    dfAirPorts = dfAirPorts.limit(10)

#Append table.
tableNamesArray.append({'AIRPORTS':dfAirPorts})

dfAirPorts.show()

In [ ]:
#
#  ------  USATEMP  TABLE
#

dfTemps = spark.read.options(header=True, delimiter=',', inferSchema='True').csv(config.globalLandTemp_file)

#Filter for US only and removes any rows where Average Temp does not exist.
dfTemps = dfTemps.filter( (dfTemps.Country == 'United States') & (dfTemps.AverageTemperature != 'Nan') )

#Splits date into year, month, and day of week. Lower cases state to make it easy to join.
dfTemps = dfTemps.withColumn('year', year(dfTemps.dt)) \
                 .withColumn('month', month(dfTemps.dt))\
                 .withColumn('dayOfweek', dayofweek(dfTemps.dt))\
                 .withColumn('State', lower (dfTemps.State)) 

#Create an unique ID for the rows. Rounds Tempature to 3 decimel places.
dfTemps = dfTemps.withColumn("id", concat(translate(dfTemps["dt"], "-" , ""), dfTemps.State )) \
                 .withColumn("AverageTemperature", round("AverageTemperature", 3)) \
                 .withColumn("AverageTemperatureUncertainty", round("AverageTemperatureUncertainty", 3))

dfTemps = dfTemps.drop("dt", "Country")

if config.limitDFtoTen:
    dfTemps = dfTemps.limit(10)
    
#Append table.
tableNamesArray.append({'USATEMP':dfTemps})

In [ ]:
#
#  ------  USADEMOGRAPHICS  TABLE
#
dfCityDemo = spark.read.options(header=True, delimiter=';').csv(config.usDemograph_file)

#Lower cases columns to be used easily when joining.
dfCityDemo = dfCityDemo.withColumn('City', lower(dfCityDemo.City)) \
                        .withColumn('State', lower(dfCityDemo.State)) \
                        .withColumn('count', col("count").cast(IntegerType()) ) 

#Pivots Race to reduce duplicated rows with only race and count being different.
dfCityDemo = dfCityDemo.groupBy("City", "State", "Median Age", "Male Population", "Female Population", "Total Population", "Number of Veterans", "Foreign-born", "Average Household Size", "State Code") \
                        .pivot("Race") \
                        .sum("Count") 

#Renames columns to lower case.
#casts columns into appropriate type.
dfCityDemo = dfCityDemo.select( 
                                col('City').alias('city'),
                                col('State').alias('state'),
                                col('State Code').alias('state_code'), 
                                col('Median Age').alias('median_age').cast(DoubleType()), 
                                col('Male Population').alias('male_population').cast(IntegerType()), 
                                col('Female Population').alias('female_population').cast(IntegerType()), 
                                col('Total Population').alias('total_population').cast(IntegerType()), 
                                col('Foreign-born').alias('foreign_born').cast(IntegerType()), 
                                col('Average Household Size').alias('average_household_size').cast(DoubleType()), 
                                col('American Indian and Alaska Native').alias('native').cast(IntegerType()), 
                                col('Asian').alias('asian').cast(IntegerType()),
                                col('Black or African-American').alias('black_african_america').cast(IntegerType()), 
                                col('Hispanic or Latino').alias('hispanic_latino').cast(IntegerType()),
                                col('White').alias('white').cast(IntegerType())
                                )


#Verifies that data that exists in Port Code table is inserted only.
dfCityDemo.createOrReplaceTempView("USADEMOGRAPHICS")
dfCityDemo = spark.sql("SELECT * FROM USADEMOGRAPHICS WHERE (state_code, city) in (select statecode, municipality from PORTCODES)")

if config.limitDFtoTen:
    dfCityDemo = dfCityDemo.limit(10)

tableNamesArray.append({'USADEMOGRAPHICS':dfCityDemo})

dfCityDemo.show()


In [ ]:
#
#  ------  IMMIGRATION  TABLE
#
dfImmigration = spark.read.options(inferSchema='True').parquet(config.immig_file)

#Filter dataset to mode 1 which is airplane.
dfImmigration = dfImmigration.filter(dfImmigration["i94mode"] == 1)

#Cast into Int
dfImmigration = dfImmigration \
                .withColumn("arrdate", regexp_replace(dfImmigration["arrdate"], '\..*$', '').cast(IntegerType()) ) \
                .withColumn("depdate", regexp_replace(dfImmigration["depdate"], '\..*$', '').cast(IntegerType()) )

#Used in the next sql.
dfImmigration.createOrReplaceTempView("dateTable")

#Adds arrdate and depdate to the year 1960.
dfImmigration = spark.sql("""
                    SELECT T1.*, 
                    date_add('1960-01-01', arrdate ) as arrival_full,
                    day(date_add('1960-01-01', arrdate ) ) as arrival_day,
                    month(date_add('1960-01-01', arrdate ) ) as arrival_month,
                    year(date_add('1960-01-01', arrdate ) ) as arrival_year,
                    date_add('1960-01-01', depdate ) as dep_full,
                    day(date_add('1960-01-01', depdate) ) as dep_day,
                    month(date_add('1960-01-01', depdate) ) as dep_month,
                    year(date_add('1960-01-01', depdate) ) as dep_year
                    FROM dateTable T1 
                    """)


dfImmigration = dfImmigration.select(
                                    col('cicid').cast(IntegerType()),
                                    col('i94cit').alias('city').cast(IntegerType()),
                                    col('i94res').alias('residence').cast(IntegerType()),
                                    col('i94port').alias('port').cast(IntegerType()),
                                    col('i94addr').alias('address').cast(IntegerType()),
                                    col('i94bir').alias('age').cast(IntegerType()),
                                    col('i94visa').alias('visa').cast(IntegerType()),
                                    col('dtadfile'),
                                    col('visapost'),
                                    col('biryear').cast(IntegerType()),
                                    col('dtaddto'), col('gender'), col('airline'), 
                                    col('fltno'), col('visatype'), col('arrival_full'), 
                                    col('arrival_day'), col('arrival_month'), col('arrival_year'), 
                                    col('dep_full'), col('dep_day'), col('dep_month'), col('dep_year')
                                    )



if config.limitDFtoTen:
    dfImmigration = dfImmigration.limit(10)

tableNamesArray.append({'IMMIGRATION':dfImmigration})

dfImmigration.show()

In [ ]:
#Writes all Dataframes into Redshift.
for tables in tableNamesArray:
    [[key, value]] = tables.items()
    redShiftWrite(value, key)

In [ ]:
#Confirms data has been inserted into the tables.
for tables in tableNamesArray:
    [[key, value]] = tables.items()
    currentTable = redShiftRead(spark, key)
    if currentTable.count() == value.count():
        print("Count correct on table : " + key)
    else:
        print("Count incorrect on table : " + key)